In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import cv2

In [2]:
df = pd.read_csv('labels.csv')
df.head()

,filepath,xmin,xmax,ymin,ymax
0,./images/vehicle_66.xml,45,113,197,238
1,./images/vehicle_72.xml,246,301,323,352
2,./images/vehicle_99.xml,849,1045,643,743
3,./images/vehicle_156.xml,2043,2631,1768,2100
4,./images/vehicle_142.xml,976,1240,851,986


In [3]:
import xml.etree.ElementTree as xet

In [4]:
filename = df['filepath'][0]
filename

'./images/vehicle_66.xml'

In [5]:
def getFilename(filename):
    filename_image = xet.parse(filename).getroot().find('filename').text
    filepath_image = os.path.join('./images',filename_image)
    return filepath_image

In [6]:
getFilename(filename)

'./images/vehicle_66.jpg'

In [7]:
image_path = list(df['filepath'].apply(getFilename))
image_path 

['./images/vehicle_66.jpg',
 './images/vehicle_72.jpg',
 './images/vehicle_99.jpg',
 './images/vehicle_156.jpg',
 './images/vehicle_142.jpg',
 './images/vehicle_143.jpg',
 './images/vehicle_157.jpg',
 './images/vehicle_98.jpg',
 './images/vehicle_73.jpg',
 './images/vehicle_67.jpg',
 './images/vehicle_59.jpg',
 './images/vehicle_71.jpg',
 './images/vehicle_65.jpg',
 './images/vehicle_141.jpg',
 './images/vehicle_155.jpg',
 './images/vehicle_169.jpg',
 './images/vehicle_168.jpg',
 './images/vehicle_154.jpg',
 './images/vehicle_140.jpg',
 './images/vehicle_64.jpg',
 './images/vehicle_70.jpg',
 './images/vehicle_58.jpg',
 './images/vehicle_74.jpg',
 './images/vehicle_60.jpg',
 './images/vehicle_48.jpg',
 './images/vehicle_178.jpg',
 './images/vehicle_144.jpg',
 './images/vehicle_150.jpg',
 './images/vehicle_151.jpg',
 './images/vehicle_145.jpg',
 './images/vehicle_179.jpg',
 './images/vehicle_49.jpg',
 './images/vehicle_61.jpg',
 './images/vehicle_75.jpg',
 './images/vehicle_63.jpg',
 './

#### verify image and output

In [8]:
file_path = image_path[0]
file_path 

'./images/vehicle_66.jpg'

In [ ]:
img = cv2.imread(file_path)

cv2.namedWindow('example',cv2.WINDOW_NORMAL)
cv2.imshow('example',img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
# 1093	1396	645	727
cv2.rectangle(img,(1093,645),(1396,727),(0,255,0),3)
cv2.namedWindow('example',cv2.WINDOW_NORMAL)
cv2.imshow('example',img)
cv2.waitKey(0)
cv2.destroyAllWindows()

### Data Preprocessing

In [9]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import load_img, img_to_array

2023-11-16 00:49:57.765255: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [10]:
labels = df.iloc[:,1:].values

In [11]:
labels

array([[  45,  113,  197,  238],
       [ 246,  301,  323,  352],
       [ 849, 1045,  643,  743],
       [2043, 2631, 1768, 2100],
       [ 976, 1240,  851,  986],
       [ 419,  597,  704,  800],
       [ 991, 1332, 1090, 1222],
       [ 448,  630,  661,  755],
       [ 207,  310,   90,  160],
       [ 173,  251,  184,  228],
       [ 246,  315,  224,  274],
       [ 158,  264,  277,  330],
       [ 204,  294,  162,  264],
       [1826, 2311, 1350, 1628],
       [ 400,  586,  555,  657],
       [ 669,  905,  766,  890],
       [1767, 2192, 1226, 1459],
       [ 529,  701,  603,  701],
       [ 454,  681,  711,  844],
       [ 119,  335,  244,  354],
       [ 312,  386,  296,  335],
       [ 128,  200,  209,  240],
       [  54,  131,  134,  187],
       [ 111,  186,  281,  322],
       [ 288,  396,  263,  300],
       [ 657,  829,  707,  802],
       [ 807, 1028,  644,  749],
       [1315, 1554,  685,  812],
       [ 665,  857,  549,  665],
       [1136, 1519, 1259, 1462],
       [ 4

In [12]:
data = []
output = []
for ind in range(len(image_path)):
    image = image_path[ind]
    img_arr = cv2.imread(image)
    h,w,d = img_arr.shape
    # prepprocesing
    load_image = load_img(image,target_size=(224,224))
    load_image_arr = img_to_array(load_image)
    norm_load_image_arr = load_image_arr/255.0 # normalization
    # normalization to labels
    xmin,xmax,ymin,ymax = labels[ind]
    nxmin,nxmax = xmin/w,xmax/w
    nymin,nymax = ymin/h,ymax/h
    label_norm = (nxmin,nxmax,nymin,nymax) # normalized output
    # -------------- append
    data.append(norm_load_image_arr)
    output.append(label_norm)

In [13]:
X = np.array(data,dtype=np.float32)
y = np.array(output,dtype=np.float32)

In [14]:
X.shape,y.shape

((180, 224, 224, 3), (180, 4))

In [15]:
x_train,x_test,y_train,y_test = train_test_split(X,y,train_size=0.8,random_state=0)
x_train.shape,x_test.shape,y_train.shape,y_test.shape

((144, 224, 224, 3), (36, 224, 224, 3), (144, 4), (36, 4))

### Deep Learning Model

In [16]:
from tensorflow.keras.applications import MobileNetV2, InceptionV3, InceptionResNetV2
from tensorflow.keras.layers import Dense, Dropout, Flatten, Input
from tensorflow.keras.models import Model
import tensorflow as tf

In [17]:
inception_resnet = InceptionResNetV2(weights="imagenet",include_top=False,
                                     input_tensor=Input(shape=(224,224,3)))
inception_resnet.trainable=False
# ---------------------
headmodel = inception_resnet.output
headmodel = Flatten()(headmodel)
headmodel = Dense(500,activation="relu")(headmodel)
headmodel = Dense(250,activation="relu")(headmodel)
headmodel = Dense(4,activation='sigmoid')(headmodel)
# ---------- model
model = Model(inputs=inception_resnet.input,outputs=headmodel)

In [18]:
# complie model
model.compile(loss='mse',optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4))
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 111, 111, 32)         864       ['input_1[0][0]']             
                                                                                                  
 batch_normalization (Batch  (None, 111, 111, 32)         96        ['conv2d[0][0]']              
 Normalization)                                                                                   
                                                                                                  
 activation (Activation)     (None, 111, 111, 32)         0         ['batch_normalization[0][0

 conv2d_11 (Conv2D)          (None, 25, 25, 64)           12288     ['average_pooling2d[0][0]']   
                                                                                                  
 batch_normalization_5 (Bat  (None, 25, 25, 96)           288       ['conv2d_5[0][0]']            
 chNormalization)                                                                                 
                                                                                                  
 batch_normalization_7 (Bat  (None, 25, 25, 64)           192       ['conv2d_7[0][0]']            
 chNormalization)                                                                                 
                                                                                                  
 batch_normalization_10 (Ba  (None, 25, 25, 96)           288       ['conv2d_10[0][0]']           
 tchNormalization)                                                                                
          

 block35_1_conv (Conv2D)     (None, 25, 25, 320)          41280     ['block35_1_mixed[0][0]']     
                                                                                                  
 custom_scale_layer (Custom  (None, 25, 25, 320)          0         ['mixed_5b[0][0]',            
 ScaleLayer)                                                         'block35_1_conv[0][0]']      
                                                                                                  
 block35_1_ac (Activation)   (None, 25, 25, 320)          0         ['custom_scale_layer[0][0]']  
                                                                                                  
 conv2d_21 (Conv2D)          (None, 25, 25, 32)           10240     ['block35_1_ac[0][0]']        
                                                                                                  
 batch_normalization_21 (Ba  (None, 25, 25, 32)           96        ['conv2d_21[0][0]']           
 tchNormal

                                                                                                  
 activation_25 (Activation)  (None, 25, 25, 32)           0         ['batch_normalization_25[0][0]
                                                                    ']                            
                                                                                                  
 activation_28 (Activation)  (None, 25, 25, 48)           0         ['batch_normalization_28[0][0]
                                                                    ']                            
                                                                                                  
 conv2d_24 (Conv2D)          (None, 25, 25, 32)           10240     ['block35_2_ac[0][0]']        
                                                                                                  
 conv2d_26 (Conv2D)          (None, 25, 25, 32)           9216      ['activation_25[0][0]']       
          

                                                                                                  
 activation_32 (Activation)  (None, 25, 25, 32)           0         ['batch_normalization_32[0][0]
                                                                    ']                            
                                                                                                  
 activation_35 (Activation)  (None, 25, 25, 64)           0         ['batch_normalization_35[0][0]
                                                                    ']                            
                                                                                                  
 block35_4_mixed (Concatena  (None, 25, 25, 128)          0         ['activation_30[0][0]',       
 te)                                                                 'activation_32[0][0]',       
                                                                     'activation_35[0][0]']       
          

...

 e)                                                                  'activation_170[0][0]']      
                                                                                                  
 block8_2_conv (Conv2D)      (None, 5, 5, 2080)           933920    ['block8_2_mixed[0][0]']      
                                                                                                  
 custom_scale_layer_31 (Cus  (None, 5, 5, 2080)           0         ['block8_1_ac[0][0]',         
 tomScaleLayer)                                                      'block8_2_conv[0][0]']       
                                                                                                  
 block8_2_ac (Activation)    (None, 5, 5, 2080)           0         ['custom_scale_layer_31[0][0]'
                                                                    ]                             
                                                                                                  
 conv2d_17

 )                                                                  ]']                           
                                                                                                  
 block8_4_mixed (Concatenat  (None, 5, 5, 448)            0         ['activation_175[0][0]',      
 e)                                                                  'activation_178[0][0]']      
                                                                                                  
 block8_4_conv (Conv2D)      (None, 5, 5, 2080)           933920    ['block8_4_mixed[0][0]']      
                                                                                                  
 custom_scale_layer_33 (Cus  (None, 5, 5, 2080)           0         ['block8_3_ac[0][0]',         
 tomScaleLayer)                                                      'block8_4_conv[0][0]']       
                                                                                                  
 block8_4_

 )                                                                  ]']                           
                                                                                                  
 activation_186 (Activation  (None, 5, 5, 256)            0         ['batch_normalization_186[0][0
 )                                                                  ]']                           
                                                                                                  
 block8_6_mixed (Concatenat  (None, 5, 5, 448)            0         ['activation_183[0][0]',      
 e)                                                                  'activation_186[0][0]']      
                                                                                                  
 block8_6_conv (Conv2D)      (None, 5, 5, 2080)           933920    ['block8_6_mixed[0][0]']      
                                                                                                  
 custom_sc

 atchNormalization)                                                                               
                                                                                                  
 activation_191 (Activation  (None, 5, 5, 192)            0         ['batch_normalization_191[0][0
 )                                                                  ]']                           
                                                                                                  
 activation_194 (Activation  (None, 5, 5, 256)            0         ['batch_normalization_194[0][0
 )                                                                  ]']                           
                                                                                                  
 block8_8_mixed (Concatenat  (None, 5, 5, 448)            0         ['activation_191[0][0]',      
 e)                                                                  'activation_194[0][0]']      
          

 atchNormalization)                                                                               
                                                                                                  
 batch_normalization_202 (B  (None, 5, 5, 256)            768       ['conv2d_202[0][0]']          
 atchNormalization)                                                                               
                                                                                                  
 activation_199 (Activation  (None, 5, 5, 192)            0         ['batch_normalization_199[0][0
 )                                                                  ]']                           
                                                                                                  
 activation_202 (Activation  (None, 5, 5, 256)            0         ['batch_normalization_202[0][0
 )                                                                  ]']                           
          

### model training

In [19]:
from tensorflow.keras.callbacks import TensorBoard

In [20]:
tfb = TensorBoard('object_detection')

In [21]:
history = model.fit(x=x_train,y=y_train,batch_size=10,epochs=100,
                    validation_data=(x_test,y_test),callbacks=[tfb])

Epoch 1/100
15/15 [==============================] - 67s 3s/step - loss: 0.1224 - val_loss: 0.1047
Epoch 2/100
15/15 [==============================] - 35s 2s/step - loss: 0.1069 - val_loss: 0.1192
Epoch 3/100
15/15 [==============================] - 32s 2s/step - loss: 0.0838 - val_loss: 0.0824
Epoch 4/100
15/15 [==============================] - 33s 2s/step - loss: 0.0743 - val_loss: 0.0697
Epoch 5/100
15/15 [==============================] - 34s 2s/step - loss: 0.0736 - val_loss: 0.0827
Epoch 6/100
15/15 [==============================] - 38s 3s/step - loss: 0.0710 - val_loss: 0.0573
Epoch 7/100
15/15 [==============================] - 34s 2s/step - loss: 0.0587 - val_loss: 0.0585
Epoch 8/100
15/15 [==============================] - 31s 2s/step - loss: 0.0524 - val_loss: 0.0570
Epoch 9/100
15/15 [==============================] - 30s 2s/step - loss: 0.0489 - val_loss: 0.0657
Epoch 10/100
15/15 [==============================] - 29s 2s/step - loss: 0.0511 - val_loss: 0.0637
Epoch 11/

15/15 [==============================] - 28s 2s/step - loss: 0.0356 - val_loss: 0.0466
Epoch 84/100
15/15 [==============================] - 27s 2s/step - loss: 0.0301 - val_loss: 0.0240
Epoch 85/100
15/15 [==============================] - 27s 2s/step - loss: 0.0116 - val_loss: 0.0149
Epoch 86/100
15/15 [==============================] - 31s 2s/step - loss: 0.0122 - val_loss: 0.0148
Epoch 87/100
15/15 [==============================] - 29s 2s/step - loss: 0.0087 - val_loss: 0.0098
Epoch 88/100
15/15 [==============================] - 27s 2s/step - loss: 0.0068 - val_loss: 0.0103
Epoch 89/100
15/15 [==============================] - 28s 2s/step - loss: 0.0052 - val_loss: 0.0092
Epoch 90/100
15/15 [==============================] - 31s 2s/step - loss: 0.0024 - val_loss: 0.0090
Epoch 91/100
15/15 [==============================] - 29s 2s/step - loss: 0.0022 - val_loss: 0.0098
Epoch 92/100
15/15 [==============================] - 28s 2s/step - loss: 0.0023 - val_loss: 0.0096
Epoch 93/100


In [22]:
history = model.fit(x=x_train,y=y_train,batch_size=10,epochs=200,
                    validation_data=(x_test,y_test),callbacks=[tfb],initial_epoch=101)

Epoch 102/200
15/15 [==============================] - 39s 3s/step - loss: 2.0935e-04 - val_loss: 0.0084
Epoch 103/200
15/15 [==============================] - 37s 2s/step - loss: 2.6065e-04 - val_loss: 0.0084
Epoch 104/200
15/15 [==============================] - 31s 2s/step - loss: 2.8801e-04 - val_loss: 0.0077
Epoch 105/200
15/15 [==============================] - 32s 2s/step - loss: 3.1146e-04 - val_loss: 0.0078
Epoch 106/200
15/15 [==============================] - 30s 2s/step - loss: 2.6697e-04 - val_loss: 0.0077
Epoch 107/200
15/15 [==============================] - 32s 2s/step - loss: 5.9937e-04 - val_loss: 0.0082
Epoch 108/200
15/15 [==============================] - 29s 2s/step - loss: 0.0014 - val_loss: 0.0094
Epoch 109/200
15/15 [==============================] - 35s 2s/step - loss: 0.0016 - val_loss: 0.0082
Epoch 110/200
15/15 [==============================] - 32s 2s/step - loss: 5.3018e-04 - val_loss: 0.0076
Epoch 111/200
15/15 [==============================] - 30s 2s/s

15/15 [==============================] - 30s 2s/step - loss: 4.3353e-04 - val_loss: 0.0080
Epoch 181/200
15/15 [==============================] - 31s 2s/step - loss: 2.7243e-04 - val_loss: 0.0082
Epoch 182/200
15/15 [==============================] - 30s 2s/step - loss: 2.0175e-04 - val_loss: 0.0078
Epoch 183/200
15/15 [==============================] - 30s 2s/step - loss: 3.6793e-04 - val_loss: 0.0076
Epoch 184/200
15/15 [==============================] - 33s 2s/step - loss: 2.7144e-04 - val_loss: 0.0076
Epoch 185/200
15/15 [==============================] - 33s 2s/step - loss: 2.2076e-04 - val_loss: 0.0075
Epoch 186/200
15/15 [==============================] - 30s 2s/step - loss: 2.3699e-04 - val_loss: 0.0080
Epoch 187/200
15/15 [==============================] - 31s 2s/step - loss: 1.8451e-04 - val_loss: 0.0075
Epoch 188/200
15/15 [==============================] - 32s 2s/step - loss: 2.2521e-04 - val_loss: 0.0081
Epoch 189/200
15/15 [==============================] - 29s 2s/step - 

In [23]:
model.save('./models/object_detection.h5')

/Users/fasihmuhammad/anaconda3/lib/python3.11/site-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
